# Namensregister
In diesem Notebook erstellen wir ein Namensregisters ``dictionary`` in Form einer Liste von Listen. 
Die Lister auf oberster Ebene soll für jeden Buchstaben a-z einen Eintrag d.h. eine Liste enthalten. In jeder Liste pro Eintrag stehen die sortierten Namen die mit einem bestimmten Buchstaben anfnagen, also:

``dictionary = [['Adrian', 'Anna', ...], ['Ben', 'berta'], ...]``

Wir lesen diese Vornamen aus einer CSV-Datei, welche doppelte Einträge enthält. Wir werden Schritt für Schritt Funktionen schreiben die uns helfen mit dem Namensregister bequem umzugehen.

**Lerninhalt**

+ Lesen einer CSV-Datei
+ Transformation von Zeichenketter in Kleinbuchstaben
+ Entwicklung einer Hashfunktion
+ Füllen des Namensregisters ``dictionary`` (mithilfe der Hashfunktion)
+ Löschen ``unique`` von doppelten Einträgen aus **unsortierter** Namensliste
+ Erzeugen einer Alphabet-Generators ``alphabet_range``
+ Test ob Name in Namensregister enthalten ``contains``, d.h. in **unsortierter** Namensliste
+ Selbstimplementierter Vergleichsoperator ``is_smaller`` für Zeichenketten
+ Schnelles sortieren der Namenslisten durch ``merge_sort``
+ Initialisierung des Namensregisters (mit doppelten Einträgen) in eine Funktion zusammenfassen um ``merge_sort`` und ``unique_sorted`` bzw. ``unique_contains`` zu testen
+ Löschen ``unique_sorted`` von doppelten Einträgen aus **sortierter** Namensliste
+ Test ob Name in Namensregister enthalten ``unique_contains``, d.h. in **sortierter** Namensliste. Hierbei implementieren wir die binäre Suche ``binary_search``
+ Überall kleine Tests einbauen


**Lerninhalt & Lernziel**

+ Umgang mit Listen (erzeugen, füllen, ändern, durch iterieren)
+ Vor- aber auch Nachteile einer Hashmap und Hashfunktion verstehen
+ Vorteile der Sortierung erkennen, d.h. auch ein Gefühl für die Laufzeitunterschiede entwickeln
+ Rekursion nutzen
+ Funktionen erster Ordnung (Funktion als Argument)
+ Binäre Suche, als auch die logarithmische Laufzeit $\mathcal{O}(\log(n))$ verstehen
+ Testen schätzen lernen

----

## Aufgabe 0

importieren Sie ``reader`` aus dem modul ``csv``.

In [109]:
from csv import reader

## Aufgabe 1

Lesen Sie die Vornamen aus der folgenden CSV-Datei ``'./data/baby-names.csv'`` und fügen Sie die Namen in die Liste ``names`` ein.

Diese Datei enthält die Top 1000 Mädchen- und Jungennamen von 1880 bis 2009 in den USA. 
Der Header lautet ``"year","name","percent","sex"``. ``year`` ist das jeweilige Jahr, ``name`` der Vorname, ``percent`` der prozentuale Anteil der Neugeborenen mit diesem Namen und ``sex`` das Geschlecht (``'boy', 'girl'``). 
Die Datei ist sortiert nach ``year`` dann nach ``percent``.
Da jedoch ein Name für mehrere Jahre enthalten ist, entsteht eine **unsortierte** Liste.

In [110]:
max_printed_names = 10
names = []
with open('baby-names.csv') as file:
    babynames = reader(file, delimiter=',')
    for row in babynames:
        names.append(row[1])

In [111]:
names[:max_printed_names]

['name',
 'John',
 'William',
 'James',
 'Charles',
 'George',
 'Frank',
 'Joseph',
 'Thomas',
 'Henry']

## Aufgabe 2

1. Finden Sie heraus, wie Python Zeichenketten vergleicht z.B. was ergibt ``'Anna' <= 'Annaa'``.
2. Konstruieren Sie eine Hashfunktion ``hash_str``. Ihre Hashfunktion erzeugt aus einem String eine ganzzahligen positive Zahl, dabei soll Klein- und Großschreibung ignoriert werden, d.h.: ``hash_str('Anna') == hash_str('anNA)``.
3. Testen Sie ihre Hashfunktion 

**Tipp:** Erzeugen Sie sich erst eine Hilfsfunktion ``lowercase``, welche die Großbuchstaben einer Zeichenkette in Kleinbuchstaben umwandelt. Für die gesamte Aufgabe könnten die Python-Funktionen ``ord`` und ``chr`` hilfreich sein!

In [112]:
# Python scheint alpabethisch zu vergleichen, jedoch unter Berücksichtigung der Groß- und Kleinschreibung
print('Anna' <= 'Aanna')
print('Anna' <= 'Bene')
print('Paul' > 'Carolin')
print('Anna' > 'Aanna')
print('anna' == 'AnNA')

False
True
True
True
False


In [113]:
# Unicode convertion
print(ord('A'))
print(ord('B'))
print(ord('a'))
print(ord('b'))

65
66
97
98


In [114]:
def lowercase(string):
    diff = ord('a') - ord('A')
    result = ''
    for c in string:
        code = ord(c)
        lc = c
        if code <= ord('Z') and code >= ord('A'):
            lc = chr(code + diff)
        result += lc
    return result

In [115]:
# Schnelltest von lowercase()
lowercase('AbCTddd')

'abctddd'

In [116]:
# Natürlich gibt es diese Funktion bereits in Python
'AbCTddd'.lower()

'abctddd'

In [117]:
def hash_str(string):
    return ord(lowercase(string[0]))-ord('a')

In [118]:
# Test der Hashfunktion!
test_names = ['Anna', 'Bene', 'Claudia', 'Dominik', 'Erika', 
              'Felix', 'Gerta', 'Hermine', 'Iris', 'Janus', 'Klaus', 
              'Lena', 'Markus', 'Nils', 'Otto', 'Petra', 'Quinn', 
              'Ria', 'Steffen', 'Tina', 'Ulrich', 'Vikki', 'Winz', 
              'Xavier', 'Yesenia', 'Zachary']

for name in test_names:
    print(f'{name} |-> {hash_str(name)}')

Anna |-> 0
Bene |-> 1
Claudia |-> 2
Dominik |-> 3
Erika |-> 4
Felix |-> 5
Gerta |-> 6
Hermine |-> 7
Iris |-> 8
Janus |-> 9
Klaus |-> 10
Lena |-> 11
Markus |-> 12
Nils |-> 13
Otto |-> 14
Petra |-> 15
Quinn |-> 16
Ria |-> 17
Steffen |-> 18
Tina |-> 19
Ulrich |-> 20
Vikki |-> 21
Winz |-> 22
Xavier |-> 23
Yesenia |-> 24
Zachary |-> 25


## Aufgabe 3

Nutzen Sie nun Ihre Hashfunktion ``hash_str`` um das Namensregisters ``dictionary`` zu bauen, d.h.
1. Erstellen Sie ein leeres Namensregister ``dictionary``
2. Füllen Sie es
3. Lassen Sie sich ein Namensliste eines Buchstabens anzeigen

**Tipp:** Mit ``dictionary[hash_str('Q')]`` sollten Sie die Namensliste der Namen die mit ``'Q'`` oder ``'q'`` beginnen addressieren können.

In [119]:
# Namensregister initialisieren, benutzten von syntaktischem Zucker ;)
dictionary = [[] for _ in range(26)]

# Achtung: dies hier funktioniert nicht, da nur eine Liste angelegt wird und ihre Referenz kopiert wird
# dictionary = [[]] * 26

# oder ohne Zucker
dictionary = []
for _ in range(26):
    dictionary.append([])

In [120]:
# Namensregister mit Namen aus der CSV füllen
for name in names:
    dictionary[hash_str(name)].append(name)

In [121]:
dictionary[hash_str('Q')][:max_printed_names]

['Quincy',
 'Quincy',
 'Quincy',
 'Quincy',
 'Quincy',
 'Quincy',
 'Quincy',
 'Quincy',
 'Quincy',
 'Quincy']

## Aufgabe 4

Schreiben Sie eine Funktion ``unique``, welche doppelte Einträge aus Ihrem Namensregister löscht.

In [122]:
# Verwenden der Methode aus 'Karten sortieren' oder neu implementieren z.B.

# Falle, folgender Code funktioniert nicht:
# for bucket in dictionary:
#     bucket = list(set(bucket))
def unique(dictionary):
    for i in range(len(dictionary)):
        dictionary[i] = list(set(dictionary[i]))
        
unique(dictionary)

In [123]:
dictionary[hash_str('Q')]

['Qiana',
 'Quintin',
 'Quentin',
 'Quinton',
 'Quinn',
 'Quinten',
 'Queenie',
 'Queen',
 'Quint',
 'Quincy',
 'Quiana']

## Aufgabe 5

Es wäre praktisch einen alphabetischen ``range`` sagen wir ``alph_range(start, end)`` zu haben, welcher uns Buchstaben von einem Startbuchstaben bis zu einem Endbuchstaben zurückgibt z.B. ``alph_range(0, 26)`` wäre eine Sequenz (``generator``) von ``'a'`` bis ``'z'``.
Aufauend darauf wäre eine Methode ``alphabet_range(lower_case=True)`` gut, welche entweder eine Sequenz (``generator``) des kleinen oder großen Alphabets zurückgibt.
1. Schreiben Sie die Funktion ``alph_range``
2. Schreiben Sie die Funktion ``alphabet_range`` bei diese ``alph_range`` benutzt
3. Testen Sie ``alphabet_range`` indem Sie sich ``list(alphabet_range())`` und ``list(alphabet_range(False))`` anzeigen lassen

In [124]:
def alph_range(start, end):
    alph = ord('a') + start
    for i in range(start,end):
        yield chr(alph)
        alph = alph + 1

In [125]:
# Ausgabe aller Kleinbuchstaben
text = ''
for c in alph_range(0, 26):
    text += (' ' + c)
print(text)

 a b c d e f g h i j k l m n o p q r s t u v w x y z


In [126]:
# Ausgabe aller Großbuchstaben
text = ''
start = -(ord('a')-ord('A'))
end = start + 26
for c in alph_range(start, end):
    text += (' ' + c)
print(text)

 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z


In [127]:
# Warum nicht in eine Methode kapseln
def alphabet_range(lower_case=True):
    start = 0
    end = 26
    if not lower_case:
        start = -(ord('a')-ord('A'))
        end = start + 26 
    return alph_range(start, end)

In [128]:
print(list(alphabet_range()))

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [129]:
print(list(alphabet_range(False)))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


## Aufgabe 6

Schreiben Sie eine Funktion ``contains(dictionary, name)``, welche genau dann ``True`` zurückgibt, wenn Ihr Namensregister den Namen ``name`` enthält.

In [130]:
# Python's in ist hier sehr mächtig, alternativ wäre auch eine Schleife korrekt (gerade für Anfänger)
def contains(dictionary, name):
    return name in dictionary[hash_str(name)]

# Alternative, nutzt nicht alle Python-Features
def contains(dictionary, name):
    for element in dictionary[hash_str(name)]:
        if element == name:
            return True
    return False

In [131]:
contains(dictionary, 'Quincy')

True

In [132]:
contains(dictionary, 'Quincyy')

False

In [133]:
# contains ist case sensitive
contains(dictionary, 'quincy')

False

## Aufgabe 7
Als nächstes Wollen wir die Namensliste also ``dictionary[hash_str('a')]`` bis ``dictionary[hash_str('z')]`` sortieren. Hierfür brauchen wir einen Ordnung auf unseren Vornamen oder anders gesagt einen Vergleichsoperator ``is_smaller`` und einen Sortieralgorithmus wobei wir diesmal Merge-Sort ``merge_sort`` implementieren.
1. Schreiben Sie eine Funktion ``is_smaller(name1, name2)`` die ``True`` zurückgibt genau dann wenn ``name1`` in der Namensliste vor ``name2`` stehen soll. **Extra:** Sie könnten auf den Python Zeichenkettenvergleich verzichten und ihn durch Ihren Code ersetzten.
2. Implementieren Sie ``merge_sort`` **Tipp:** Schreiben Sie eine Hilfsmethode ``merge`` die zwei sortierte Listen zusammenfügt (das Resultat ist erneut eine sortierte Liste).
3. Implementieren Sie die Funktion ``sort_dictionary`` welche Ihr gesamtes Namensregister sortiert.
4. Welchen Vorteil könnte es haben ein sortiertes Namensregister zu haben?
5. Testen Sie alle Ihre Funktionen

In [134]:
# zunächst brauchen wir einen Vergleichsoperator, hier eine Version die auf den Zeichenkettenvergleich von 
# von Python verzichtet
def is_smaller(name1, name2):
    name1_lower = lowercase(name1)
    name2_lower = lowercase(name2)
    for i in range(min(len(name1_lower), len(name2_lower))):
        if ord(name1_lower[i]) < ord(name2_lower[i]):
            return True
        elif ord(name1_lower[i]) > ord(name2_lower[i]):
            return False
    # at this point the prefix of both strings are equals
    return len(name1) < len(name2)

# oder
def is_smaller(name1, name2):
    return lowercase(name1) < lowercase(name2)

# oder
def is_smaller(name1, name2):
    return name1.lower() < name2.lower()

In [135]:
# In plain Python wir haben:
print('Anna' < 'Aanna')
print('Anna' < 'Bene')
print('Carolin' < 'Paul')
print('Aanna' < 'Anna')
print('Anna' < 'Annaa')

print('Anna' < 'aanna')
print('Anna' < 'bene')
print('Carolin' < 'paul')
print('Aanna' < 'anna')

False
True
True
True
True
True
True
True
True


In [136]:
# und nun:
print(is_smaller('Anna','Aanna'))
print(is_smaller('Anna','Bene'))
print(is_smaller('Carolin', 'Paul'))
print(is_smaller('Aanna', 'Anna'))

print(is_smaller('Anna','aanna'))
print(is_smaller('Anna','bene'))
print(is_smaller('Carolin', 'paul'))
print(is_smaller('Aanna', 'anna'))

False
True
True
True
False
True
True
True


In [137]:
# als nächstes implementieren wir das Verschmelzen zweier sortierter Listen
def merge(sorted_list1, sorted_list2, is_smaller):
    # we assume both lists are sorted (ascending)
    result = []
    i = 0
    j = 0
    while i < len(sorted_list1) and j < len(sorted_list2):
        if is_smaller(sorted_list1[i], sorted_list2[j]):
            result.append(sorted_list1[i])
            i += 1
        else:
            result.append(sorted_list2[j])
            j += 1
    rest = sorted_list2
    index = j
    if i < len(sorted_list1)-1:
        rest = sorted_list1
        index = i
    
    # append the remaining content
    for k in range(index,len(rest)):
        result.append(rest[k])
    
    return result

In [138]:
# Schnelltest
sorted_list1 = [1,2,3,4,5,6]
sorted_list2 = [8,9,20,21]

merge1 = merge(sorted_list1, sorted_list2, lambda a, b: a < b)
merge2 = merge(sorted_list2, sorted_list1, lambda a, b: a < b)
print(merge1 == merge2)
print(len(merge1) == len(sorted_list1) + len(sorted_list2))
print(merge1)

True
True
[1, 2, 3, 4, 5, 6, 8, 9, 20, 21]


In [139]:
# nun folgt der merge_sort Algorithmus
def merge_sort(mylist, is_smaller):
    if len(mylist) <= 1:
        return mylist
    if len(mylist) == 2:
        if(is_smaller(mylist[1], mylist[0])):
            return [mylist[1], mylist[0]]
        else:
            return mylist
    else:
        pivot = int(len(mylist) / 2)
        list1 = mylist[:pivot]
        list2 = mylist[pivot:]
        sorted_list1 = merge_sort(list1, is_smaller)
        sorted_list2 = merge_sort(list2, is_smaller)
        # time to merge
        return merge(sorted_list1, sorted_list2, is_smaller)

In [140]:
# Schnelltest
mylist = [1,6,4,3,9,2, 123, -12, 8, 0, 1, 1, -12, 564]
sorted_list = merge_sort(mylist, lambda a, b : a < b)

print(all(ele in sorted_list for ele in mylist))
print(all(ele in mylist for ele in sorted_list))
print(len(sorted_list) == len(mylist))

True
True
True


In [141]:
def sort_dictionary(dictionary):
    for i in range(len(dictionary)):
        dictionary[i] = merge_sort(dictionary[i], is_smaller)

sort_dictionary(dictionary)
dictionary[hash_str('Q')]

['Qiana',
 'Queen',
 'Queenie',
 'Quentin',
 'Quiana',
 'Quincy',
 'Quinn',
 'Quintin',
 'Quinton']

## Aufgabe 8

Um unsere Methoden zu testen wäre es gut das Namensregister mit einer Funktion neu zu erzeugen, d.h. unsortiert und mit doppelten Einträgen.
Definieren Sie eine Funktion ``init_dictionary`` welche dies übernimmt. Sie müssen im Endeffekt nur den Code von oben richtig kombinieren.

In [142]:
def init_dictionary():
    # read from file
    names = []
    with open('baby-names.csv') as file:
        babynames = reader(file, delimiter=',')
        for row in babynames:
            names.append(row[1])
    
    # generate empty dictionary
    dictionary = [[] for _ in range(26)]
    
    # fill the dictionary
    for name in names:
        dictionary[hash_str(name)].append(name)
    
    # dont forget to return it
    return dictionary

## Aufgabe 9

Schreiben Sie neue Varianten von ``unique`` (``unique_sorted``) und ``contains`` (``contains_sorted``) unter der Annahme, dass Ihr Namensregister sortiert ist, d.h. nutzen Sie diese Tatsache aus! 
**Tipp:** ``unique_sorted`` ähnelt ``merge``, und ``contains_sorted`` beruht auf der binären Suche.

In [143]:
def unique_sorted(dictionary):
     for i in range(len(dictionary)):
        if len(dictionary[i]) > 0:
            result = []
            result.append(dictionary[i][0])
            for j in range(1,len(dictionary[i])):
                if(dictionary[i][j] != result[-1]):
                    result.append(dictionary[i][j])
            dictionary[i] = result

In [144]:
dictionary_fresh = init_dictionary()

In [145]:
dictionary = dictionary_fresh.copy()

In [146]:
dictionary[0][:max_printed_names]

['Arthur',
 'Albert',
 'Andrew',
 'Alfred',
 'Alexander',
 'August',
 'Allen',
 'Archie',
 'Alex',
 'Anthony']

In [147]:
dictionary[25][:max_printed_names]

['Zack', 'Zeb', 'Zeke', 'Zack', 'Zeb', 'Zack', 'Zeb', 'Zack', 'Zeb', 'Zeke']

In [148]:
# Für den Test erzeugen wir ein frisches sortiertes Namesregister
sort_dictionary(dictionary)
dictionary[hash_str('Q')][:max_printed_names]

['Qiana',
 'Qiana',
 'Qiana',
 'Qiana',
 'Qiana',
 'Queen',
 'Queen',
 'Queen',
 'Queen',
 'Queen']

In [149]:
unique_sorted(dictionary)
dictionary[hash_str('A')][:max_printed_names]

['Aaden',
 'Aaliyah',
 'Aarav',
 'Aaron',
 'Ab',
 'Abagail',
 'Abb',
 'Abbey',
 'Abbie',
 'Abbigail']

In [150]:
def binary_search(names, name, is_smaller):
    pivot = int(len(names) / 2)
    if names[pivot] == name:
        return pivot
    if len(names) <= 1:
        return None
    if is_smaller(name, names[pivot]):
        return binary_search(names[:pivot], name, is_smaller)
    else:
        right = binary_search(names[pivot:], name, is_smaller)
        if right == None:
            return None
        return pivot + right

In [151]:
mylist = [1,2,3,4,5,6,7,8]

print(mylist[binary_search(mylist, 7, lambda a, b: a < b)] == 7)
print(mylist[binary_search(mylist, 1, lambda a, b: a < b)] == 1)
print(mylist[binary_search(mylist, 8, lambda a, b: a < b)] == 8)
print(binary_search(mylist, 18, lambda a, b: a < b))

True
True
True
None


In [152]:
def unique_contains(dictionary, name):
    result = binary_search(dictionary[hash_str(name)], name, is_smaller)
    return result != None

In [153]:
unique_contains(dictionary, 'Quinttt')

False

In [154]:
unique_contains(dictionary, 'Quint')

True

## Aufgabe 10
Schreiben Sie ein Funktion ``test_unique_contains`` die ``True`` zurückgibt, falls ``unique_contains`` für alle Namen im Namensregister ``True`` zurückgibt, andernfalls soll ``test_unique_contains`` ``False`` zurückgeben. Schreiben Sie hierfür eine kleine Hilfsfunktion ``get_names`` die Ihnen aus dem sortierten und ohne doppelten Einträgen gefüllten Namensregister ``dictionary`` alle Namen in einer Liste zurückgibt.

Ist die neue Liste die Ihnen ``get_names`` zurückgibt sortiert? Warum bzw. warum nicht?

In [159]:
def get_names(dictionary):
    result = []
    for names in dictionary:
         result += names
    return result

In [160]:
all_names = get_names(dictionary)
all_names[:max_printed_names]

['Aaden',
 'Aaliyah',
 'Aarav',
 'Aaron',
 'Ab',
 'Abagail',
 'Abb',
 'Abbey',
 'Abbie',
 'Abbigail']

In [161]:
def test_unique_contains(dictionary):
    names = get_names(dictionary)
    for name in names:
        if not unique_contains(dictionary, name):
            return False
    return True

# oder mit syntaktischem Zucker:
def test_unique_contains(dictionary):
    return all(unique_contains(dictionary, name) for name in get_names(dictionary))

In [162]:
test_unique_contains(dictionary)

True